# Volkswagen Navarra

In [19]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

## Costes iniciales

In [1022]:
dias_año=300000/1438
dias_año/2

104.3115438108484

In [24]:
purgas=140140
cambio_color=1262183
#spot_repair=47293
mantenimiento_cabina=(26646*4)+35550+35550+38524+32577+38524+26646+38524+29603

In [25]:
coste_anual=purgas+cambio_color+mantenimiento_cabina
coste_anual

1784405

In [26]:
coste_diario=coste_anual/dias_año
coste_diario

11610.743846153848

## Creacion vector inicial

In [47]:
n2a=pd.read_excel('N2A.xlsx')
n2b=pd.read_excel('N2B.xls')

In [48]:
n2a['Dato MDS nº1'].value_counts()

8E    315
0A    287
0Q    276
2T    202
P8     82
4M     67
0Z     46
W9     39
6U     17
5W      2
Name: Dato MDS nº1, dtype: int64

In [49]:
n2b['Dato MDS nº1'].value_counts()

5W    528
6U    360
0Q    354
2T    277
0A      6
P8      3
8E      2
W9      1
Name: Dato MDS nº1, dtype: int64

In [947]:
color=[]
carrocerias=[]

In [971]:
for nu in range(num):
    carrocerias.append('7231')
print(len(carrocerias))


2856


In [994]:
for nu in range(num):
    color.append('P8')
print(len(color))

2854


In [1135]:
color=np.array(color)
color=color.T
carrocerias=np.array(carrocerias).T
carrocerias=carrocerias.reshape(2860,)
color=color.reshape(2860,)

In [1136]:
entrada=pd.DataFrame()

In [1137]:
entrada['color']=color
entrada['carro']=carrocerias

In [1138]:
entrada

,color,carro
0,2T,3111
1,2T,3111
2,2T,3111
3,2T,3111
4,2T,3111
...,...,...
2855,2T,7231
2856,5W,6121
2857,0A,6221
2858,0Q,3111


In [1118]:
l_a=['0Q','2T','0A','0Z','P8','W9','4M','8E']
l_b=['0Q','2T','5W','6U']

l_as=['0A','0Z','P8','W9','4M','8E']
l_bs=['5W','6U']
l_bn=['0Q','2T']
valores=[i for i in range(180)]
linea_a=[]
cambio=[]
#entra2.shape[0]>0

## Simulacion


In [1183]:

def simulacion(l_as,l_bs,ramal):
    inicial=int(256*0.4)
    primera=0
    lin_a=[]
    lin_b=[]
    entra2=ramal
    cambio=[]
    cambio_color={'8E': 0, '0Z': 0, '0A': 0, 'P8': 0, '4M': 0, 'W9': 0, '2T': 0, '0Q': 0,'5W': 0, '6U': 0}
    new=True
    stop=True
    ult_a=True
    while(stop):
        if primera==0:
            mirada=entra2.iloc[:inicial]
            #print(mirada)
            primera=1
        else:
            if entra2.shape[0]>1: 
                mirada=entra2.iloc[[0,1]]
            else:
                #print(lin_a.shape)
                new=False
            #print(mirada)
        if new:
            entra2=entra2.drop(range(mirada.shape[0])).reset_index().drop(columns='index')
            lin_an=[mirada.iloc[i] for i in range(mirada.shape[0]) if mirada.iloc[i]['color'] in l_as ]

            if primera==1:
                lin_a.extend(lin_an)
            else:
                lin_a=lin_a.append(lin_an).reset_index().drop(columns=['index'])

            inde=[i for i in range(mirada.shape[0]) if mirada.iloc[i]['color'] in l_as ]
            mirada2=mirada.drop(inde).reset_index().drop(columns=['index'])

            lin_bn=[mirada2.iloc[i] for i in range(mirada2.shape[0]) if mirada2.iloc[i]['color'] in l_bs ]
            if primera==1:
                lin_b.extend(lin_bn)
            else:
                lin_b=lin_b.append(lin_bn).reset_index().drop(columns=['index'])
            inde2=[i for i in range(mirada2.shape[0]) if mirada2.iloc[i]['color'] in l_bs ]
            mirada3=mirada2.drop(inde2).reset_index().drop(columns=['index'])
            # Añadimos los blancos y negros
            if primera==1:
                lin_a.extend([mirada3.iloc[i] for i in range(int((mirada3.shape[0]/2))) ])
                lin_b.extend([mirada3.iloc[int((mirada3.shape[0]/2))+i] for i in range(int(mirada3.shape[0]/2)+mirada3.shape[0]%2)])
            else:
                if ult_a :
                    lin_a=lin_a.append([mirada3.iloc[i] for i in range(int((mirada3.shape[0]/2))) ])
                    lin_b=lin_b.append([mirada3.iloc[int((mirada3.shape[0]/2))+i] for i in range(int(mirada3.shape[0]/2)+mirada3.shape[0]%2)])
                    ult_a=False
                else :
                    lin_b=lin_b.append([mirada3.iloc[i] for i in range(int((mirada3.shape[0]/2))) ])
                    lin_a=lin_a.append([mirada3.iloc[int((mirada3.shape[0]/2))+i] for i in range(int(mirada3.shape[0]/2)+mirada3.shape[0]%2)])
                    ult_a=True

        # Los mayores los mandamos a la sala de pintura y actualizamos inventario
        if primera==1:
            inventario_a=dict(Counter(np.array(lin_a)[:,0]))
            inventario_b=dict(Counter(np.array(lin_b)[:,0]))
            siguiente_a=max(inventario_a, key=inventario_a.get)
            siguiente_b=max(inventario_b, key=inventario_b.get)
            lin_a=pd.DataFrame(lin_a).reset_index().drop(columns=['index'])
            lin_b=pd.DataFrame(lin_b).reset_index().drop(columns=['index'])
        else:
            lin_b=lin_b.reset_index().drop(columns=['index'])
            lin_a=lin_a.reset_index().drop(columns=['index'])
            if len(lin_an)>0:
                if len(lin_an)==1:
                    inventario_a[lin_an[0][0]]+=1
                    if not(ult_a) and mirada3.shape[0]!=0:
                        if mirada3.shape[0]==2:
                            inventario_a[mirada3[0]['color']]+=1
                            inventario_b[mirada3[1]['color']]+=1
                        else:
                            inventario_a[mirada3['color'][0]]+=1
                else:
                    inventario_a[lin_an[0][0]]+=1
                    inventario_a[lin_an[1][0]]+=1
            if len(lin_bn)>0:
                if len(lin_bn)==1:
                    inventario_b[lin_bn[0][0]]+=1
                    if ult_a and mirada3.shape[0]!=0:
                        if mirada3.shape[0]==2:
                            inventario_b[mirada3[0]['color']]+=1
                            inventario_a[mirada3[1]['color']]+=1
                        else:
                            inventario_b[mirada3['color'][0]]+=1

                else:
                    inventario_b[lin_bn[0][0]]+=1
                    inventario_b[lin_bn[1][0]]+=1


        if inventario_a[siguiente_a]==0 and primera!=1:
            siguiente_a=max(inventario_a, key=inventario_a.get)
        if inventario_b[siguiente_b]==0 and primera!=1:
            siguiente_b=max(inventario_b, key=inventario_b.get)
        #fuera_a=inventario_a[siguiente_a]

        for i in range(lin_a.shape[0]):
            if lin_a.iloc[i]['color']==siguiente_a:
                lin_a=lin_a.drop(i).reset_index().drop(columns=['index'])
                break
        for i in range(lin_b.shape[0]):
            if lin_b.iloc[i]['color']==siguiente_b:
                lin_b=lin_b.drop(i).reset_index().drop(columns=['index'])
                break

        inventario_a[siguiente_a]-=1
        inventario_b[siguiente_b]-=1

        #fuera_b=inventario_b[siguiente_b]
        #lin_b=[lin_b[i] for i in range(len(lin_b)) if lin_b[i]['color']!=siguiente_b ]

        if primera==1:
            #print('primera')
            siguiente_a_a=siguiente_a
            siguiente_b_a=siguiente_b
            primera=2

        if siguiente_a!=siguiente_a_a:
            cambio_color[siguiente_a_a]+=1
            siguiente_a_a=siguiente_a
            #print('cambio')
        if siguiente_b!=siguiente_b_a:
            cambio_color[siguiente_b_a]+=1
            siguiente_b_a=siguiente_b
            #print('cambio')

        cambio.append(siguiente_a)
        cambio.append(siguiente_b)
        #print(inventario_a)
        if (sum(inventario_a.values())==0) or (sum(inventario_b.values())==0):
            stop=False
    return cambio_color
    
    

    
    

## Costes

In [1180]:
def coste_totales(cambios,l_as,l_bs,l_c):
    coste=0
    coste_man=0
    costes_cambio={'8E': 13.5, '0Z': 21.1, '0A': 26.7, 'P8': 66.9, '4M': 35.7, 'W9': 31.6, '2T': 9.2, '0Q': 10.2,'5W': 24.6, '6U': 16}
    costes_purga={'8E': 9950, '0Z': 19960, '0A': 3988, 'P8': 63210, '4M': 33765, 'W9': 29897, '2T': 5777, '0Q': 6416,'5W': 23245, '6U': 10072}
    costes_mantenimiento={'8E': 35550, '0Z': 32577, '0A': 35550, 'P8': 38524, '4M': 38524, 'W9': 29603, '2T': 26646, '0Q': 26646,'5W': 38524, '6U': 26646}
    costes_fijos=22174

    for color,valor in cambios.items():
        coste_parcial=costes_cambio[color]*valor
        coste+=coste_parcial
    coste_anual=coste*208
    for color in l_as:
        coste_man+=(costes_mantenimiento[color]+costes_purga[color])
    for color in l_bs:
        coste_man+=(costes_mantenimiento[color]+costes_purga[color])
    for color in l_c:
        coste_man+=(costes_mantenimiento[color]+costes_purga[color])*2
    l_a=['0A','0Z','P8','W9','4M','8E']
    l_b=['5W','6U']
    l_cc=['2T','0Q']



    list_difference = list(set(l_as).difference(set(l_a)).union(set(l_bs).difference(set(l_b))))
    list_difference=[num for num in list_difference if num not in l_cc]
    coste_taller=len(list_difference)*22174
    

    return coste_anual+coste_man+coste_taller
    

In [1172]:
def gen_random(entrada):
    c=entrada['carro'].sample(frac=0.5)
    co=entrada['color'].sample(frac=0.5)
    entra=pd.DataFrame()
    entra['color']=co
    entra['carro']=c
    entra=entra.reset_index().drop(columns=['index'])
    return entra